![alt text](img/MIoT_ML.png "MIoT_ML")
# Unidad 03. Regresión lineal de una variable

El objetivo de esta práctica es comprender los fundamentos del entrenamiento de un modelo de ML simple. En particular, por su simplicidad, nos centraremos en un modelo de regresión lineal de una sola variable. 

En la vida real es muy improbable que tengáis que implementar modelos desde cero, ya que existen una gran variedad de librerías de ML disponibles, pero enfrentarse a la implementación de los conceptos más básicos os ayudará a entender cómo funcionan internamente los modelos, y ver que no existe "magia" en el ML.

**Importante**: este Notebook está desarrollado con fines docentes, y no con el objetivo de optimizar el proceso. En el desarrollo y entrenamiento de modelos de ML (offline) es vital aprovecharnos de la vectorización para la optimización de recursos (operaciones en matrices o vectores completos en lugar de procesar los elementos individualmente mediante bucles). La vectorización es clave, pero también dificulta el aprendizaje, ya que hace menos legible el código.


El Notebook contiene varios ejercicios sencillos. Debéis desarrollarlos durante la clase y enviarlos por el aula virtual del curso, en la tarea correspondiente.

## Referencias útiles para la práctica
1. API Matplotlib: [https://matplotlib.org/stable/api/index](https://matplotlib.org/stable/api/index)
2. API Numpy: [https://numpy.org/doc/stable/reference/](https://numpy.org/doc/stable/reference/)
3. Dataset para el ejercicio: [https://www.kaggle.com/datasets/camnugent/california-housing-prices](https://www.kaggle.com/datasets/camnugent/california-housing-prices).
4. [Curso](https://www.coursera.org/learn/machine-learning-course/home/welcome) de Machine Learning de Anxdrew NG en Coursera.
   * Esta unidad contiene algunos ejemplos inspirados y/o adaptados de algunas de las prácticas del curso de Machine Learning de Andrew NG


## 1. Modelo de regresión lineal de una variable

Un modelo de regresión lineal de una variable busca establecer una relación lineal entre dos variables: una variable independiente (input) y una variable dependiente (output).  El objetivo es encontrar la línea recta que mejor se ajuste a los datos, permitiendo predecir el valor de la variable dependiente a partir de la variable independiente.

La función del modelo que representa la regresión lineal simple se muestra a continuación:



$$ f_{w,b}(x^{(i)}) = wx^{(i)} + b \tag{1}$$

Esta función tiene dos parámetros $w$ y $b$ y asigna un valor "y" (output) en función de una variable de entrada "x" (input). El ajuste de los parámetros $w$ y $b$ se realiza durante el proceso de entrenamiento del modelo.

## 2. Conjunto de datos
### 2.1. Cargar y seleccionar los datos

Usaremos el conjunto de datos que hemos visto en las unidades anteriores, pero, en este caso, solo nos centraremos en dos variables, para ajustarnos a las condiciones de la regresión lineal. En la Unidad 01 vimos que entre la variable a predecir *median_house_value* y *median_income* había una correlación lineal bastante alta. EL objetivo de esta Unidad será desarrollar una regresión lineal entre esas 2 variables.


**Recordad** que durante la Unidad 01 descubrimos que la *median_income* era una representación escalada de la mediana de los ingresos de los hogares en una manzana, cuyo valor representaba decenas de miles de dolares (ej. 35=350.000$). Las dos variables están en escalas muy diferentes. Para facilitar las gráficas y el seguimiento de la práctica, realizaremos un escalado sobre la característica *median_house_value* dividiendo su valor por 1.000.

**Nota**: aunque ya tenemos los datos preprocesados de la Unidad 02, vamos a trabajar con los datos originales para que su interpretación sea más sencilla. 




In [ ]:
try:
    import kagglehub
except ImportError as err:
    !pip install kagglehub
    import kagglehub
    
try:
    from rich import print
except ImportError as err:
    !pip install rich
    from rich import print

try:
    import pandas as pd
except ImportError as err:
    !pip install pandas
    import pandas as pd

try:
    from sklearn.model_selection import train_test_split
except ImportError as err:
    !pip install sklearn
    from sklearn.model_selection import train_test_split

try:
    import matplotlib.pyplot as plt
except ImportError as err:
    !pip install matplotlib
    import matplotlib.pyplot as plt

# Descarga del dataset
path = kagglehub.dataset_download("camnugent/california-housing-prices")
print("Path to dataset files:", path)
dataset = pd.read_csv(path+"/housing.csv") # Carga datos desde un CSV y devuelve un DataFrame 

# Generación de los datasets de entrenamiento y test
SEED=1234 # Semilla para asegurarnos que siempre se generen los mismos valores aleatorios
trainset, testset = train_test_split(dataset, test_size=0.3, train_size=0.7, random_state=SEED, shuffle=True, stratify=dataset["ocean_proximity"])

# Para esta Unidad solo nos interesan las variables "median_income" y "median_house_value"
trainset = trainset[["median_income","median_house_value" ]].copy()
testset = testset[["median_income","median_house_value" ]].copy()

# En unidades anteriores habíamos descubierto que "median_house_value" tenía un límite superior
# que agrupaba todas las manzanas donde la mediana del precio de la vivienda superaba dicho límite
# para no condicionar el modelo, esas observaciones son eliminadas
# Eliminación de las observaciones con  median_house_value == max
# Eliminación de observaciones de Entrenamiento
trainset = trainset[trainset["median_house_value"] != dataset["median_house_value"].max()]
# Eliminación de observaciones de Test
testset = testset[testset["median_house_value"] != dataset["median_house_value"].max()]

# Por motivos docentes y de visualización
# escalamos "median_house_value" en el mismo rango que "median_income"
# Ver *Nota* superior
trainset["median_house_value"] = trainset["median_house_value"].apply(lambda X: X/1000)
testset["median_house_value"] = testset["median_house_value"].apply(lambda X: X/1000)

# Ploteamos los datos para visualizar la correlación entre las dos variables
trainset.plot(x="median_income", y="median_house_value", kind="scatter", alpha=0.1)
plt.show()
print("Correlación entre ambas variables")
print(trainset.corr())


### 2.2. Modelo básico a partir de 2 puntos

Empezaremos con lo más básico, cogiendo 2 puntos de los datos de entrenamiento para construir un modelo entre ellos. Vamos a trabajar desde la base, es decir, empleando [arrays](https://numpy.org/doc/stable/reference/generated/numpy.array.html) de numpy.

**Nota**: Las observaciones seleccionadas están deliberadamente escogidas para acercarse a la recta que mejor describe la relación entre ambas variables.


In [ ]:
try:
    import numpy as np
except ImportError as err:
    !pip install numpy
    import numpy as np

# En este punto podríamos seleccionar varias observaciones del conjunto de entrenamiento
# Nuestro caso de uso no se puede representar fácilmente con una regresión lineal entre solo 2 variables
# Por motivos docentes y para poder mostrar mejor los ejercicios
# he seleccionado deliberadamente 2 observaciones presentes en el dataset
#datos_seleccionados = pd.DataFrame(data={'median_income': [1.0298, 1.9891], 'median_house_value': [300, 500]})
datos_seleccionados = pd.DataFrame(data={'median_income': [2, 4], 'median_house_value': [92.6, 202.6]})
pto1 = datos_seleccionados.iloc[0] # Devuelve una Serie de Pandas con la fila 0
pto2 = datos_seleccionados.iloc[1] # Devuelve una Serie de Pandas con la fil 1

# Almacenamos en un array las entradas (inputs o 'x'), es decir, los valores de "median_income"
x_train = np.array([pto1.loc["median_income"], pto2.loc["median_income"]])

# Almacenamos en un array las salidas (outputs, labels, o 'y'), es decir, los valores de "median_house_value"
y_train = np.array([pto1.loc["median_house_value"], pto2.loc["median_house_value"]])

# se podría haber hecho directamente sin calcular los puntos con
#x_train = np.array(datos_seleccionados["median_income"], dtype=float)
#y_train = np.array(datos_seleccionados["median_house_value"], dtype=float)
              
# Mostramos los puntos escogidos
print(f"x_train = {x_train}")
print(f"y_train = {y_train}")


#### Notación
- Usaremos $m$ para denotar el número de ejemplos que tenemos para entrenar, es decir, el número de observaciones de nuestro conjunto de entrenamiento (2 en este caso). Podemos emplear <code>len()</code> para obtener el valor o el método [shape()](https://numpy.org/doc/stable/reference/generated/numpy.shape.html) de los arrays de numpy.


- Usaremos la notación (x$^{(i)}$, y$^{(i)}$) para denotar la  $i$-esima observación del conjunto de entrenamiento.

    -    **Recordad** que en Python (y también en Numpy) los índices empiezan en cero.


In [ ]:
print(f"Número de observaciones del entrenamiento: {len(x_train)}")
i = 0 # i-esima observación
x_i = x_train[i]
y_i = y_train[i]
print(f"La observación {i} es (x^({i}), y^({i})) = ({x_i}, {y_i})")

Visualicemos en un gráfico los datos de las observaciones seleccionadas.

In [ ]:
# Plotea los puntos
plt.figure()

plt.scatter(trainset["median_income"], trainset["median_house_value"], alpha=0.1)
plt.scatter(x_train, y_train, marker='x', c='r', label="Valores reales")

plt.title("Mediana del valor de la vivienda") # Especifica el título
plt.ylabel('median house value') # Añade el eje y
plt.xlabel('median income') # Añade el eje x
plt.legend()
plt.show()

## 3. Modelo de regresión lineal de una variable

Recordamos la función del modelo para la regresión lineal de una sola variable:

$$ f_{w,b}(x^{(i)}) = wx^{(i)} + b \tag{1}$$

Esta función solo permite representar líneas rectas (es la ecuación de una recta). Cambiando los valores $w$ y $b$ generamos diferentes rectas, que se podrán ajustar mejor o peor a nuestros datos (el objetivo del entrenamiento es precisamente encontrar los mejores valores para $w$ y $b$). Lógicamente, no podemos escoger unos valores de $w$ y $b$ diferentes para cada observación, sino que necesitamos encontrar los valores que mejor se ajustan, de media, a todas las observaciones del conjunto de entrenamiento.


Si escogemos 2 valores cualesquiera para  $w$ y $b$ podremos usar nuestro modelo para predecir la $y$ en base a las entradas $x$.

- Para $x^{(0)}$, `f_wb = w * x[0] + b`
- Para $x^{(1)}$, `f_wb = w * x[1] + b`




In [ ]:
# esta función recibe las 'x' de todas las observaciones, y devuelve todas las 'y' (las predicciones)
def calcula_salida_modelo(x, w, b):
    m = len(x)
    predicciones = np.zeros(m) # genera un array de ceros con numpy, de la longitud del resultado
    for i in range(m):
        # función del modelo que se aplica sobre los datos de entrada para unos valores w, b concretos (recibidos como parámetros)
        predicciones[i] = w * x[i] + b 
    return predicciones

# esta función dibuja las predicciones (una línea azul por 'plot') y los valores reales (x rojas por 'scatter')
def plot_resultados_modelo(inputs, predicciones, labels):
    plt.clf() # reset de la figura 
    # Genera un plot con las predicciones
    plt.plot(inputs, predicciones, c='b', label='Predicciones')
    # Añade al plot los puntos reales de entrenamiento
    plt.scatter(inputs, labels, marker='x', c='r', label='Valores reales')

    plt.title("Valor medio de la vivienda")  # Añade un título al gráfico
    plt.ylabel('median house value') # Añade el eje y
    plt.xlabel('median income') # Añade el eje x
    plt.legend()
    plt.show()


# seleccionamos valores aleatorios para w y b como punto de partida (los resultados serán malos)
w = 100 
b = 100
print(f"w: {w}, b: {b}")

predicciones = calcula_salida_modelo(x_train, w, b)  # calculamos las predicciones
for i, (x, y, pred) in enumerate(zip(x_train, y_train, predicciones)):
    print(f"x({i})={x} - valor real: {y} valor predicho: {pred:.2f}")

plot_resultados_modelo(x_train, predicciones, y_train)  # dibujamos las predicciones y los valores reales



Vemos que las predicciones (azul) están lejos de los valores reales (x rojas). Lógico, la recta ($w$,$b$) fue elegida al azar.

**Ejercicio**

Prueba a jugar con los valores de $w$ y $b$ para ver cómo varía la recta hasta conseguir unos valores que ajusten bien.



In [ ]:
##### modifica w y b hasta generar la recta que une los dos puntos
w = 100
b = 100

predicciones = calcula_salida_modelo(x_train, w, b)
print(f"w: {w}, b: {b}")
for i, (x, y,pred) in enumerate(zip(x_train, y_train, predicciones)):
    print(f"x({i})={x} - valor real: {y} valor predicho: {pred:.2f}")
plot_resultados_modelo(x_train, predicciones, y_train)



**EJERCICIO 1 PARA ENTREGAR EN EL AULA VIRTUAL**

La función "calcula_salida_modelo" definida anteriormente de forma tradicional puede ser simplificada usando un estilo de programación más propio de Python, haciendo uso de sus capacidades para procesar listas y arrays. Porporciona otra versión de la función "calcula_salida_modelo" que sea más sintética.


In [ ]:
### EJERCICIO 1 PARA ENTREGAR EN EL AULA VIRTUAL 
# Escribe otra versión de la función calcula_salida_modelo (mostrada a continuación) que sea más sintética





Una vez escogemos unos valores para los parámetros de nuestro modelo que ajustan bien con los datos de entrenamiento, podemos usar el modelo  para predecir con nuevos datos con los que nunca se entrenó. A continuación mostramos las prediciones para dos entradas con sus respectivos valores reales, y vemos que una se aproxima bastante (3.4), mientras que la otra no tanto (4.4). 

In [ ]:
# fijamos los valores de w y b que hemos calculado a mano, y que sí se ajustan a los puntos seleccionados
w = 55
b = -17

x_test = np.array([3.4, 4.4]) # probamos con otros valores reales del dataset
y_test = np.array([155.5, 174.1])  

y_prediccion = w * x_test + b  # calculamos las predicciones para esos dos valores

# imprimimos el resultado para los nuevos puntos, devolviendo sus valores a la escala original
print(f"Precio de una vivienda en el que la familia tiene un ingreso de {x_test[0]*10000}$: {y_prediccion[0]*1000}$")
print(f"Precio de una vivienda en el que la familia tiene un ingreso de {x_test[1]*10000}$: {y_prediccion[1]*1000}$")

trainset.plot(x="median_income", y="median_house_value", kind="scatter", alpha=0.1)
plt.scatter(x_train, y_train, marker='x', c='r', label="Puntos iniciales") # valor real de los puntos iniciales  
plt.scatter(x_test, y_test, marker='o', c='r', label='Valores reales') # valor real de los nuevos puntos
plt.scatter(x_test, y_prediccion, marker='o', c='cyan', label='Valores predichos')  # valor predicho para los nuevos puntos
plt.legend()
plt.show()


## 4. Función de coste

### 4.1. Definición de la función de coste

Tenemos una función con 2 parámetros  ($w$ y $b$)  que debemos definir para ajustar dicho modelo pero, 
**¿cómo escogemos los valores $w$ y $b$?**
$$ f_{w,b}(x^{(i)}) = wx^{(i)} + b \tag{1}$$


Obviamente, no podemos estar probando con todas las posibles combinaciones "a mano". Necesitamos entrenar el modelo, y que se ajusten automáticamente. Para poder decidir que una combinación es mejor que otra, es necesario poder establecer un **coste** de la combinación. De esta forma, podremos comparar combinaciones y escoger entre las diferentes opciones. 


De forma general, podemos representar la función de coste para los párametros $w$ y $b$ como: $J(w,b)$


Existen diferentes funciones de coste pero una habitual en ML es el *Mean Squared Error* (MSE).

$$MSE=\frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})^2\tag{2}$$

y particularmente, en problemas de regresión,  es habitual introducir un factor $\frac{1}{2}$, quedando la función de coste como: 

  $$J(w,b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})^2 \tag{3}$$ 

 donde:
 -  $f_{w,b}(x^{(i)}) = wx^{(i)} + b $ (función del modelo).
 - $f_{w,b}(x^{(i)})$ es nuestra predicción para la entrada $x^i$ empleando unos parámetros ($w,b$) específicos. Por simplicidad, es habitual representar la predicción como: $\hat{y}^i$
- $(f_{w,b}(x^{(i)}) -y^{(i)})^2$ es la diferencia al cuadrado entre el valor real y la predicción.   
- Todas esas diferencias a lo largo de las $m$ observaciones son sumadas y divididas por `2m` para producir el coste final, $J(w,b)$. 


**Fijaos** que el coste se calcula sobre todas las observaciones de entrenamiento, por lo que el objetivo es obtener los parámetros $w,b$ que mejor se comportan para todas las observaciones.

 **Importante**: Cuanto más cerca estén las predicciones de los valores reales, menor coste. **Debemos buscar una forma de minimizar la función de coste** para obtener la mejor solución.


$$\underset{w, b}{\text{minimize }} J(w,b)\tag{4}$$

In [ ]:
def calculo_coste(x, y, w, b): 
    """
    Calcula la función de coste para la regresión lineal
    
    Args:
      x (ndarray (m,)): Entradas, m observaciones 
      y (ndarray (m,)): salidas
      w,b (scalar)    : parámetros del modelo  
    
    Returns
        coste_total (float)
    """
    # observaciones
    m = len(x)
    cost_sum = 0 
    for i in range(m): # recorremos todas las observaciones y sumamos su coste
        pred = w * x[i] + b   
        cost_sum += (pred - y[i]) ** 2    
    return (1 / (2 * m)) * cost_sum # promediamos el coste 


# Comprobamos los diferentes costes que tendríamos para el mismo conjunto de entrada
# cuando variamos los parámetros w, b, probando con (50,50), (100,100) y (150,150)
# Podéis probar con otras configuraciones
w_pruebas = [50,100,150] # posibles valores para w
b_pruebas = [50,100,150] # posibles valores para b

print("Para los datos de entrenamiento:")
for i,(x,y) in enumerate(zip(x_train, y_train)):
    print(f"(x^{i},y^{i})=({x},{y})")

for w, b in zip(w_pruebas, b_pruebas):
    print(f"El coste total con w={w} y b={b} es: {calculo_coste(x_train, y_train, w, b):.2f}")


### 4.2. Visualización de la función de coste 

Para mejorar la comprensión de la función de coste, y en qué consiste su minimización, vamos a simplificar el problema y suponer que el valor de $b$ es fijo y que solo tenemos que  buscar el valor óptimo de $w$; es decir, nuestra función solo tiene un parámetro "libre". Para cada posible valor de $w$, podemos calcular su función de coste ($J(w)$) y graficar dicho valor. A medida que vayamos calculando valores y graficándolos, iremos dibujando la curva del coste para cada $w$. 


Si solo tenemos 1 variable "libre" y le damos todos los posibles valores, la función de coste que podríamos pintar con los resultados tendría  forma convexa con un mínimo global. Nuestro objetivo minimizando dicha función de coste ($J(w)$) es encontrar dicho mínimo global.  Necesitamos un algoritmo que nos permita movernos por la función de coste hasta alcanzar el mínimo.

Veamos la representación gráfica de esta explicación. Fijaos en la figura inferior cómo evoluciona el coste (parte derecha) si movemos el valor de $w$ (parte izquierda). 
- La figura de la parte derecha refleja en azul la forma de la función de coste si graficásemos todos sus valores (es una aproximación). También se muestra el coste de una configuración concreta (valor de $w$) dentro de la función de coste (el punto rojo). El objetivo es conseguir que el coste concreto (el punto rojo) se situe en el mínimo de la función (esto se produce alrededor de w=55, como habíamos visto)
- La figura de la parte izquierda muestra el valor de cada predicción (línea azul) respecto al valor real (puntos rojos). Cuanto más lejos esté el valor de la predicción del valor real, mayor coste.

**Nota**: Podéis jugar con el valor fijo de $b$.

**Nota 2**: el gráfico es interactivo, podéis modificar el valor de $w$. El refresco puede ser algo lento.

In [ ]:
import sys
import os
try:
    from lab_utils_uni import plt_intuition, plt_stationary, plt_update_onclick
except ModuleNotFoundError:  # en el caso del Colab, no estarán los ficheros auxiliares
    repo_path = "/content/MIOT_ML"
    if not os.path.exists(repo_path):
      !git clone http://github.com/dmerap/MIOT_ML  # clonamos el repositorio en el Colab para usar los ficheros auxiliares
    else:
      print("El repositorio ya existe")
    import sys
    sys.path.append(repo_path)
    from lab_utils_uni import plt_intuition, plt_stationary, plt_update_onclick

plt.close()
# Esta función está en un fichero de Python con métodos útiles para la práctica
# No es necesario que la entendáis. Se emplea solo por motivos docentes
plt_intuition(x_train, y_train, fixed_b=-17)

Aumentemos la dificultad y dejemos "libres" los 2 parámetros $w$ y $b$. En este caso, para poder graficar la función de coste, necesitamos un gráfico en 3 dimensiones o emplear un gráfico de contornos (aka [isolíneas](https://es.wikipedia.org/wiki/Isol%C3%ADnea)). 

- El gráfico de la derecha (gráfico de contornos) nos permite seleccionar una combinación de los parámetros $w$ y $b$. Vemos que el centro de los contornos se haya alrededor de $w$=55 y $b$=-17, como habíamos visto.
- El gráfico de la izquierda nos permite visualizar la recta y el coste de la selección de parámetros actual.



In [ ]:
try:
    import ipympl
except ImportError as err:
    !pip install ipympl
    import ipympl

## START-Estas líneas se necesitan para ejecutar el Notebook en Colab
## IMPORTANTE: la primera vez que se instala ipympl en Colab el gráfico no es interactivo
## Debéis ejecutar nuevamente la celda
try:
    from google.colab import output
    print("El Notebook se está ejecutando en Colab")
    output.enable_custom_widget_manager()
except ModuleNotFoundError:
    print("El Notebook no se está ejecutando en Colab")

%matplotlib widget

plt.close('all') 
fig, ax, dyn_items = plt_stationary(x_train, y_train)
print("")
updater = plt_update_onclick(fig, ax, x_train, y_train, dyn_items)

## 5. Gradiente descendente

### 5.1. Definición

Ahora que entendemos cómo funciona una regresión lineal y cómo se calcula el coste de los posibles valores ($w,b$), debemos encontrar una forma de buscarlos de forma automática. Está claro que no podemos ir probando todas las posibles combinaciones de forma manual. 

¿Cómo minimizamos la función de coste? Empleando el algoritmo de **descenso de gradiente**. 

$$\underset{w, b}{\text{minimize }}J(w,b)\tag{4}$$


Este algoritmo define la forma de actualizar  los parámetros $w$, $b$ empleando la derivada parcial de la función de coste respecto de sus variables ($w$ y $b$). Esta derivada indica la dirección y magnitud del cambio necesario para reducir el error.

$$\begin{align*} \text{Repetir}&\text{ hasta que converja:} \; \lbrace \newline
\;  w &= w -  \alpha \frac{\partial J(w,b)}{\partial w} \tag{5}  \; \newline 
 b &= b -  \alpha \frac{\partial J(w,b)}{\partial b}  \newline \rbrace
\end{align*}$$

donde:
- $\alpha$ es la tasa de aprendizaje o *learning rate*.
- $\frac{\partial J(w,b)}{\partial w}$ es la derivada parcial de la función de coste respecto a $w$.
- $\frac{\partial J(w,b)}{\partial b}$ es la derivada parcial de la función de coste respecto a $b$.

La actualización de parámetros implica la derivada parcial de la función de coste respecto de $w$ y de $b$. **No es necesario que aprendais a hacer la derivada parcial**, a continuación os proporciono el resultado de dicha derivada parcial:

$$
\begin{align}
\frac{\partial J(w,b)}{\partial w}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})x^{(i)} \tag{4}\\
  \frac{\partial J(w,b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)}) \tag{5}\\
\end{align}
$$


**Importante**: Los parámetros $w$, $b$ deben ser actualizados simultaneamente. Esto significa que las derivadas parciales deben calcularse antes de actualizar ninguno de los parámetros.

Os resumo los pasos del algoritmo: 
1. Inicializamos los valores de $w$ y $b$, típicamente con valores aleatorios.
2. Calculamos las predicciones.
3. Calculamos las derivadas parciales.
4. Ajustamos los parámetros $w$ y $b$ en función del gradiente.
5. Repetimos hasta cumplir la condición de salida.



El tamaño y dirección del "paso" para actualizar los parámetros $w$ y $b$ viene marcado por el resultado de la derivada parcial:
- **Dirección**: Si la derivada es positiva, debemos disminuir el parámetro; si es negativa, debemos aumentarlo.
- **Magnitud**: Qué tan grande es el cambio necesario. Valores mayores significan que el coste cambia rápidamente en esa dirección.

El hiperparámetro $\alpha$ (tasa de aprendizaje o *learning rate*) permite aumentar o reducir el "paso" en la actualización de los parámetros. Este hiperparámetro es algo que deberemos escoger y ajustar al entrenar los modelos, y que suele ser clave:
- Un $\alpha$ pequeño puede hacer que no lleguemos a un buen ajuste final del modelo en las iteraciones que le permitamos entrenar.
-  Un $\alpha$ demasiado grande puede resultar en un entrenamieno divergente. 


Vamos a implementar el descenso de gradiente para una sola variable con la idea de entender los fundamentos.


In [ ]:
def calculo_gradiente(x, y, w, b): 
    """
    Calcula el gradiente para la regresión lineal
    Args:
      x (ndarray (m,)): datos de entrada, m ejemplos 
      y (ndarray (m,)): valores de salida
      w,b (scalar)    : parámetros del modelo  
    Returns
      dj_dw (scalar): El gradiente del coste respecto a w
      dj_db (scalar): El gradiente del coste respecto a b     
     """
    
    # observaciones
    m = len(x)    

    # valores que devolveremos inicializados a cero
    dj_dw = 0
    dj_db = 0

    # calculo de los gradientes
    for i in range(m):  
        f_wb = w * x[i] + b # cálculo de la predicción
        dj_dw_i = (f_wb - y[i]) * x[i]  # un componente del sumatorio para w
        dj_db_i = f_wb - y[i]  # un componente del sumatorio para b
        dj_db += dj_db_i # añadimos el componente al sumatorio de w
        dj_dw += dj_dw_i # añadimos el componente al sumatorio de b
    dj_dw = dj_dw / m  # paso final de dividir entre m
    dj_db = dj_db / m 
        
    return dj_dw, dj_db

Para una mejor comprensión de los gradientes y la función de coste, vamos a fijar el valor de parámetro $b$ y probar con diferentes puntos para $w$. El gradiente nos devolverá:
- **Dirección**: Si la derivada es positiva, debemos disminuir el parámetro; si es negativa, debemos aumentarlo.
- **Magnitud**: Qué tan grande es el cambio necesario. Valores mayores significan que el coste cambia rápidamente en esa dirección.

**Nota**: En el mínimo global, la derivada será 0.

In [ ]:
from lab_utils_uni import plt_gradients
plt.close('all') 
plt_gradients(x_train,y_train, calculo_coste, calculo_gradiente, fixed_b=-17, tested_points=[-200, -150, -100, -50, 0,50,100,150,200, 300])
plt.show()

### 5.2. Implementación del gradiente descendente
Una vez comprendido cómo los gradientes nos ayudarán a saber la dirección y la magnitud de las actualizaciones de nuestros parámetros $w$ y $b$, ya solo necesitamos implementar el algoritmo del gradiente descendente para automatizar el proceso.

In [ ]:
def gradiente_descendente(x, y, w_in, b_in, alpha, num_iters, cost_function, gradient_function): 
    """
    Desarrolla el algoritmo de gradiente descendente para ajustar w y b. 
    Actualiza los valores w y b realizando num_iters cálculos de gradiente
    con una tasa de aprendizaje alpha
    
    Args:
      x (ndarray (m,))  : Datos entrada, m observacines 
      y (ndarray (m,))  : valores reales de salida
      w_in,b_in (scalar): Valores iniciales de w y b  
      alpha (float):      Learning rate o tasa de aprendizaje
      num_iters (int):    número de iteraciones (épocas) para ejecutar el gradiente descendente.
      cost_function:      función que calcula el coste
      gradient_function:  función que calcula el radiente
      
    Returns:
      w (scalar): valor calculado de w una vez ejecutado el algoritmo de gradiente descendente.
      b (scalar): valor calculado de b una vez ejecutado el algoritmo de gradiente descendente.
      J_history (List): histórico de valores de coste 
      p_history (list): histórico de valores de (w,b)
      """
    
    # array para almancear los valores históricos de coste y (w,b)
    # Los usaremos para graficar
    J_history = []
    p_history = []
    b = b_in
    w = w_in
    
    for i in range(num_iters):
        # Calcula el gradiente empleando la función pasada como argumento
        dj_dw, dj_db = gradient_function(x, y, w , b)     

        # Actualiza los parámetros empleando la ecuación 5
        b = b - alpha * dj_db                            
        w = w - alpha * dj_dw                            

        ### Esta parte del algoritmo es opcional, es solo para guardar el histórico de resultados intermedios y visualizarlos
        coste = cost_function(x, y, w , b)
        J_history.append(coste ) # almacena el coste en el histórico
        p_history.append([w,b]) # almacena los valores de w y b en el histórico

        # Mostramos el avance cada 10 iteraciones
        if i%10==0:
            print(f"Iteración {i}. Coste {coste}.",
                  f"dj_dw: {dj_dw: 0.2f}, dj_db: {dj_db: 0.2f}  ",
                  f"w: {w: 0.2f}, b:{b: 0.2f}"
                 )

 
    return w, b, J_history, p_history 

Probemos el algoritmo con nuestros datos:

**Nota**: si jugáis con el número de iteraciones, o con los valores de los parámetros de entrada, conseguiréis ajustar mejor el modelo.

In [ ]:
w_init = 10 # valor inicial de w
b_init = 10 # valor inicial de b
iteraciones = 3000
alpha = 0.01

w_calculado, b_calculado, J_history, p_history = gradiente_descendente(x_train, y_train, w_init, b_init, alpha, iteraciones, calculo_coste, calculo_gradiente)


Vemos que, partiendo de  $w$=10 y $b$=10, en 3000 iteraciones, los valores finales ($w$=54,27 y $b$=-14,98) son aproximadamente los previstos.
Probemos el modelo con los resultados del algoritmo ($w$ y $b$) para los dos puntos por los que empezamos. 

In [ ]:
predicciones = calcula_salida_modelo(x_train, w_calculado, b_calculado)
plt.close('all') 
plot_resultados_modelo(x_train, predicciones,y_train)

### 5.3. Coste vs iteraciones
Un gráfico del coste frente a las iteraciones es una medida interesante para valorar el progreso en el descenso de gradiente. El coste siempre debería disminuir en ejecuciones exitosas. El cambio en el coste es muy rápido al comienzo y luego tiende a desacelerar, por lo que, a menudo,  es útil graficar el descenso inicial en una escala diferente a la del descenso final. En los siguientes gráficos, observe la escala del coste en los ejes y el paso de iteración.

In [ ]:
# plot coste vs iteraciones  
plt.close('all') 
fig, (ax1, ax2) = plt.subplots(1, 2, constrained_layout=True, figsize=(9,4))

ax1.plot(J_history[:100])
ax1.set_title("Coste vs. iteración(inicio)")
ax1.set_ylabel('Coste')
ax1.set_xlabel('Iteración')

ax2.plot(100 + np.arange(len(J_history[100:])), J_history[100:])
ax2.set_title("Coste vs. iteración (final)")
ax2.set_ylabel('Coste') 
ax2.set_xlabel('Iteración') 

plt.show()

Podemos también visualizar la evolución de los parámetros $w$ y $b$ durante la ejecución del algoritmo de gradiente descendente respecto al gráfico de contornos. Veremos como la evolución de los valores se acerca, en cada iteración, al centro del gráfico, donde se encuentra el mínimo global.

In [ ]:
from lab_utils_uni import plt_contour_wgrad
fig, ax = plt.subplots(1,1, figsize=(10, 6))
plt_contour_wgrad(x_train, y_train, p_history, ax)


**EJERCICIO 2 para entregar en el aula virtual**

En el entrenamiento previo hemos usado solamente los dos puntos seleccionados desde el inicio, de ahí los buenos resultados obtenidos respecto a la recta que los une.

Entrena ahora con todo el conjunto de entrenamiento del dataset y comenta los resultados.


In [ ]:
### EJERCICIO 2 para entregar en el aula virtual
# Escribe el código para entrenar con todo el conjunto de entrenamiento
# Ejecútalo y comenta los resultados.

